<a href="https://colab.research.google.com/github/vimalkum/Hindi-POS-Tagger/blob/master/POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)
training = open("/content/drive/My Drive/Colab Notebooks/hin_health_set02.txt","r").read()
training=training.split('\n')
sentence,tags=[],[]
#print(len(training))
for i in range(0,len(training)-1):
  sen,tag=[],[]
  #print(training[i])
  for word in training[i].split()[1:]:
    w = word.split('//')
    sen.append(w[0])
    tag.append(w[1])
    #print(w[0],w[1])
  sentence.append(sen)
  tags.append(tag)

#print(sentence)

training_data = []
for sent,tag in zip(sentence,tags):
  training_data.append((sent,tag))
                       

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
#print(word_to_ix)
tag_to_ix = {}
ix_to_tag={}
for sent, tags in training_data:
    for tag in tags:
        if tag not in tag_to_ix:
            l=len(tag_to_ix)
            tag_to_ix[tag] = l
            ix_to_tag[l]=tag
print(tag_to_ix)

{'N_NN': 0, 'PSP': 1, 'N_NNP': 2, 'RD_PUNC': 3, 'CC_CCD': 4, 'V_VM': 5, 'V_VAUX': 6, 'RP_INTF': 7, 'QT_QTC': 8, 'DM_DMD': 9, 'RB': 10, 'N_NST': 11, 'QT_QTF': 12, 'RP_RPD': 13, 'JJ': 14, 'RP_NEG': 15, 'PR_PRL': 16, 'CC_CCS': 17, 'PR_PRP': 18, 'RD_UNK': 19, 'PR_PRQ': 20, 'DM_DMQ': 21, 'DM_DMI': 22, 'PR_PRF': 23, 'RD_ECH': 24, 'RD_RDF': 25, 'DM_DMR': 26, 'QT_QTO': 27, 'PR_PRI': 28, 'RD_SYM': 29, '/RD_PUNC': 30, 'PR_PRC': 31, 'DM_DRI': 32, 'RP_INJ': 33, '/RD_SYM': 34, 'CC_CSD': 35, 'NST': 36, 'PR_RRF': 37, 'PR_RPP': 38, 'RP_RPQ': 39, 'RP_PRD': 40, 'RD_PUN': 41, 'CCS': 42, 'RP_PUNC': 43, 'CC_CS': 44, 'PR_RPL': 45, 'RD_VAUX': 46, 'N_N_NN': 47, '_VM': 48, 'RD-PUNC': 49, 'परीक्षण': 50, 'Q_QTC': 51, 'NJJ': 52, 'V_V': 53, 'RD_': 54, 'PSP,': 55, 'RD_UPNC': 56, 'N_N': 57, 'RD_RPD': 58, 'RP_PRF': 59, 'V_VAUX।': 60, 'N_NN,': 61, 'C_CCS': 62, 'RP_RP_RPD': 63, 'MD_DMD': 64, 'N_NNP,': 65, 'V_RD_PUNC': 66, 'VAUX': 67, 'PR_NEG': 68, 'RD_PUNV': 69, 'RD_PUC': 70, 'NN': 71, 'QT_QT': 72, 'JJभोजन': 73, 'V_AUX

In [0]:
# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 32
HIDDEN_DIM = 32

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

torch.manual_seed(1)

In [0]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,bidirectional=True)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        lstm_out1, _ = self.lstm(embeds.view(len(sentence),1,-1))
        tag_space = self.hidden2tag(lstm_out1.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [27]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)


with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)
training_data,testing_data = train_test_split(training_data,test_size=0.2)
for epoch in range(20):  
    for sentence, tags in training_data:
        
        model.zero_grad()

       
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

       
        tag_scores = model(sentence_in)

       
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()


with torch.no_grad():
    inputs = prepare_sequence(testing_data[0][0], word_to_ix)
    tag_scores = model(inputs)


    tag_seq=''
    print(len(tag_scores))
    for i in range(0,len(tag_scores)):
      m=max(tag_scores[i])
      for j in range(0,len(tag_scores[i])):
        if m==tag_scores[i][j]:
          print(ix_to_tag[j],end='-')
          tag_seq+=ix_to_tag[j]+'-'
print('\n')
print(tag_seq.split('-'))
print(testing_data[0][1])

#print(tag_seq)

tensor([[-4.1687, -4.5414, -4.4407,  ..., -4.2570, -4.4730, -4.3297],
        [-4.3378, -4.4665, -4.3907,  ..., -4.3426, -4.4697, -4.3296],
        [-4.2095, -4.4799, -4.3830,  ..., -4.3441, -4.5752, -4.3206],
        ...,
        [-4.0153, -4.3367, -4.3342,  ..., -4.2591, -4.5047, -4.3309],
        [-4.1266, -4.3291, -4.3457,  ..., -4.2652, -4.4511, -4.3711],
        [-4.1829, -4.3202, -4.3136,  ..., -4.4070, -4.4607, -4.4184]])
13
N_NN-PSP-N_NN-RP_NEG-V_VM-PSP-V_VM-N_NN-PSP-V_VM-V_VAUX-V_VAUX-RD_PUNC-

['N_NN', 'PSP', 'N_NN', 'RP_NEG', 'V_VM', 'PSP', 'V_VM', 'N_NN', 'PSP', 'V_VM', 'V_VAUX', 'V_VAUX', 'RD_PUNC', '']
['N_NN', 'PSP', 'N_NN', 'RP_NEG', 'V_VM', 'PSP', 'N_NN', 'N_NN', 'PSP', 'V_VM', 'V_VAUX', 'V_VAUX', 'RD_PUNC']
